In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os
import shutil
import pathlib
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.preprocessing import image



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
if os.path.exists('./train_lables') == True:
    shutil.rmtree('./train_lables')
    
train_images_folder = '../input/shopee-product-matching/train_images'
train = pd.read_csv('../input/shopee-product-matching/train.csv')

print('train shape is', train.shape )
train.head()

In [ ]:
label_names = train['label_group'].unique().tolist()

label_class_dict = {}

for i,label in enumerate(label_names):
    label_class_dict.update({label:i})

train['label_class'] = train['label_group'].copy()

train.replace({'label_class' :label_class_dict},inplace=True)

print('train shape is', train.shape )
train.head()

In [ ]:
train['class_name_temp_fmt'] = 'class_'

print('train shape is', train.shape )
train.head()

In [ ]:
train_labels_group_count_dict  = train.groupby('label_class').image.count().to_dict()
train_labels_group_dict        = train.groupby('label_class').image.agg('unique').to_dict()


**Next**, We will create the new train**['sub_folder'] **column.

sub_folder **data format **: **'class_5_18_2098400894'**

How to read **'sub_folder'** data format ?

**1-6 Positions**                        -  'class_'   (default)  - (train['class_name_temp_fmt'])
**Next Position (image_group_count)**    -  '1' or '21 (variable) -(train['image_group_count'].astype('str'))
**Next Position **                       -  '_'
**Next Position ('label_class')**        -  '0' or '1'...'11013'    train['label_class'].astype('str')
**Next Position**                        -  '_'
**Next Position ('label_group')**        -  label_group id provided by the shopee dataset 

In [ ]:
train['image_group'] =  train.label_class.map(train_labels_group_dict)
train['image_group_count'] =  train.label_class.map(train_labels_group_count_dict)

train['sub_folder'] = train['class_name_temp_fmt'] + train['image_group_count'].astype('str') + '_' + train['label_class'].astype('str') + '_'+ train['label_group'].astype('str')

print('train shape is', train.shape )
train.head()

In [ ]:
train.drop('class_name_temp_fmt',axis=1,inplace=True)

print('train shape is', train.shape )
train.head()

In [ ]:
plt.figure(figsize=(25,25))
for i in range(20):
    plt.subplot(5,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    img = image.load_img(train_images_folder+'/'+train['image'][i])
    plt.imshow(img, cmap=plt.cm.binary)
    plt.xlabel(train['sub_folder'][i])
plt.show()

**Next, ** 

We will create the follwoing data directory staructure, 

                  1. -- parent dirctory   ('../input/shopee-product-matching/)
                   1.1 -- Main_data_directory (/train_labels)
                     1.1.1 -- Sub directory   (**name format **: **'class_5_18_2098400894'**)
                        1.1.1.1 -- similar image files (Ground Truth)** (>1 images)

In [ ]:
# Parent Directory path
parent_dir          = "/kaggle/working"
main_data_directory = "train_lables"

# Path
path = os.path.join(parent_dir, main_data_directory)

# Create the directory
os.mkdir(path)

target_data_dir = path


In [ ]:
#create sub folders and copy image files from source (/train_images) to target (/kaggle/working/train_lables)

class_sub_dir_unique = []

for i in train[['sub_folder','image_group','image_group_count']][train.image_group_count>0].sub_folder:
    #copy the unique classes and image files
    if i not in class_sub_dir_unique:
        class_sub_dir_unique.append(i)
        path = os.path.join(target_data_dir, i)
        #if os.path.exists(path) == False:
        os.mkdir(path)
        for k,j in enumerate((train.image_group[train.sub_folder==i].iloc[:1]).to_list()[0]):
            img_src = train_images_folder + '/' +j
            #if os.path.exists(img_src) == False:
            shutil.copy(img_src, path)


In [ ]:
data_dir = "/kaggle/working/train_lables"

data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
class_5_18_2098400894 = list(data_dir.glob('class_5_18_2098400894/*'))
PIL.Image.open(str(class_5_18_2098400894[0]))

In [ ]:
PIL.Image.open(str(class_5_18_2098400894[1]))

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
#Visualize the data

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")